In [1]:
import qsweepy.save_pkl
import pandas as pd
import matplotlib.pyplot as plt
import qsweepy.fitting as fitting
import numpy as np

from qsweepy import *
from qsweepy.ponyfiles import *
from qsweepy.fitters.multiqubit_spectra import *

from qsweepy.ponyfiles.data_structures import *

%matplotlib inline

Failed loading module Keysight_M3102A:  No module named 'keysightSD1'
Failed loading module Keysight_M3202A:  No module named 'keysightSD1'
[WinError 126] The specified module could not be found
Failed loading module Signal_Hound_SA:  name 'signal_hound_dll' is not defined


In [2]:
parameters_dict = \
{'qubits': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'],
 'inductance_matrix_type': 'chain-nn',
 'qubit_resonator_individual': 'equal_claws',
 'qubit_qubit_coupling': 'alternating-chain-nn',
 'inductances': {'1': {'central': -0.109185896504347,
   'right': -0.0004826239072177366},
  '2': {'central': -0.11431231313261195,
   'right': -0.00027538188179635083,
   'left': 0.0008577490152635229},
  '3': {'central': -0.11131307359191432,
   'right': 0.00024345888448612619,
   'left': 0.003773033767464582},
  '4': {'central': -0.11521125144334263,
   'right': -0.0004938990389211441,
   'left': 0.0001681722604761979},
  '5': {'central': -0.11268053164777525,
   'right': -0.0011830316189549437,
   'left': 0.00039349418795583886},
  '6': {'central': -0.11537787321718858,
   'right': -0.0006438958322830763,
   'left': 2.757204098568375e-05},
  '7': {'central': -0.10241951553849499,
   'right': -0.00216192920028019,
   'left': 0.00034795371255411616},
  '8': {'central': -0.09276736882548474,
   'right': -0.0007912202614004031,
   'left': 0.0002328203609286558},
  '9': {'central': -0.10882904539008785,
   'right': -0.0018443764398353704,
   'left': -0.0001160523283298373},
  '10': {'central': -0.0975311916842425,
   'right': -0.0018302317143429473,
   'left': -0.0002249780824307208},
  '11': {'central': -0.10365991163886802, 'left': -4.374063323585438e-05}},
 'EJ1': {'1': 11816223384.264452,
  '2': 11168950261.446684,
  '3': 11815374828.756998,
  '4': 11263129085.763647,
  '5': 10981552502.675322,
  '6': 10909809335.89073,
  '7': 11666221426.430653,
  '8': 10903644122.926136,
  '9': 11563075836.984825,
  '10': 11045274424.134794,
  '11': 11265770833.103388},
 'EJ2': {'1': 1943530015.2166,
  '2': 1673183461.373451,
  '3': 1911475144.5540576,
  '4': 1769477531.8836436,
  '5': 1810726542.9254663,
  '6': 1607956688.359111,
  '7': 1988898006.2197614,
  '8': 2189188060.0125456,
  '9': 2001071976.5662835,
  '10': 1941073364.3633077,
  '11': 1945649944.4130301},
 'phi0': {'1': 0.08181626708822409,
  '2': 0.0866324181609742,
  '3': 0.10045189195678028,
  '4': 0.0863936205645889,
  '5': 0.0866026444716125,
  '6': 0.08617179042392543,
  '7': 0.07910410017305261,
  '8': 0.09000314980903502,
  '9': 0.07698168331954032,
  '10': 0.08347793740401784,
  '11': 0.07489040124974441},
 'EC': 161574581.1907917,
 'g': 586.6709956926836,
 'J1': 0.011727299185763845,
 'J2': 0.003000045037643214,
 'fr': {'1': 6260000000.0,
  '2': 6307000000.0,
  '3': 6362300000.0,
  '4': 6414000000.0,
  '5': 6458700000.0,
  '6': 6498400000.0,
  '7': 6555000000.0,
  '8': 6606000000.0,
  '9': 6658000000.0,
  '10': 6711000000.0,
  '11': 6763000000.0}}

In [10]:
sample_name = '11qubits_021119'
db = database.MyDatabase()

sweeper = sweep_extras.Sweeper(db, sample_name=sample_name)
exdir_db_inst = exdir_db.Exdir_db(db=db, sample_name=sample_name)

In [20]:
def save_parameters_dict(exdir_db_inst, parameters_dict):    
    metadata = {'inductance_matrix_type': parameters_dict['inductance_matrix_type'],
                'qubit_resonator_individual': parameters_dict['qubit_resonator_individual'],
                'qubit_qubit_coupling': parameters_dict['qubit_qubit_coupling']}
    metadata['EC'] = parameters_dict['EC']
    metadata['g'] = parameters_dict['g']
    metadata['J1'] = parameters_dict['J1']
    metadata['J2'] = parameters_dict['J2']
    
    qubits = parameters_dict['qubits']
    qubit_id_parameter = MeasurementParameter(name='qubit_id', values=[int(qubit_id) for qubit_id in qubits], 
                                              setter=False, unit='')
    coil_id_parameter = MeasurementParameter(name='coil_id', values=[int(qubit_id) for qubit_id in qubits], 
                                              setter=False, unit='')

    EJ1 = MeasurementDataset(parameters=[qubit_id_parameter], 
                             data=np.asarray([parameters_dict['EJ1'][str(qubit_id)] for qubit_id in qubit_id_parameter.values]))
    EJ2 = MeasurementDataset(parameters=[qubit_id_parameter], 
                             data=np.asarray([parameters_dict['EJ2'][str(qubit_id)] for qubit_id in qubit_id_parameter.values]))
    fr  = MeasurementDataset(parameters=[qubit_id_parameter], 
                             data=np.asarray([parameters_dict['fr'][str(qubit_id)] for qubit_id in qubit_id_parameter.values]))
    phi0= MeasurementDataset(parameters=[qubit_id_parameter], 
                             data=np.asarray([parameters_dict['phi0'][str(qubit_id)] for qubit_id in qubit_id_parameter.values]))
    L   = MeasurementDataset(parameters=[coil_id_parameter, qubit_id_parameter], data=build_inductance_matrix(parameters_dict))
    #dataset = MeasurementDataset?

    m = MeasurementState(datasets={'EJ1':EJ1, 'EJ2':EJ2, 'fr':fr, 'phi0':phi0, 'L':L}, 
                         metadata=metadata,
                         measurement_type='linear_oscillator_model_parameters',
                         sample_name=exdir_db_inst.sample_name)
    exdir_db_inst.save_measurement(m)
    return m

In [32]:
def load_parameters_dict(exdir_db_inst):    
    meas = exdir_db_inst.select_measurement(measurement_type='linear_oscillator_model_parameters')
    
    parameters_dict = {}
    parameters_dict.update(meas.metadata)
    
    L = meas.datasets['L']
    
    qubits = [str(qubit_id) for qubit_id in meas.datasets['EJ1'].parameters[0].values]
    parameters_dict['EJ1'] = {}
    parameters_dict['EJ2'] = {}
    parameters_dict['fr'] = {}
    parameters_dict['phi0'] = {}
    parameters_dict['L'] = {}
    
    for qubit_id_, qubit_id in enumerate(qubits):
        parameters_dict['EJ1'][qubit_id] = meas.datasets['EJ1'].data[qubit_id_]
        parameters_dict['EJ2'][qubit_id] = meas.datasets['EJ2'].data[qubit_id_]
        parameters_dict['fr'][qubit_id] = meas.datasets['fr'].data[qubit_id_]
        parameters_dict['phi0'][qubit_id] = meas.datasets['phi0'].data[qubit_id_]
        parameters_dict['L'][qubit_id] = {'central':meas.datasets['L'].data[qubit_id_, qubit_id_]}
        if qubit_id_ > 0:
            parameters_dict['L'][qubit_id]['left'] = meas.datasets['L'].data[qubit_id_, qubit_id_-1]
        if qubit_id_ < len(qubits)-1:
            parameters_dict['L'][qubit_id]['right'] = meas.datasets['L'].data[qubit_id_, qubit_id_+1]
        
    return parameters_dict

In [30]:
save_parameters_dict(exdir_db_inst, parameters_dict)

start: 2019-11-18 12:57:15.629097, started/done/total sweeps: 0/0/0, 
Measured data: 
'EJ1': parameters: qubit_id (),:[1, 11] (11 points) without setter
data: [1.18162234e+10 1.11689503e+10 1.18153748e+10 1.12631291e+10
 1.09815525e+10 1.09098093e+10 1.16662214e+10 1.09036441e+10
 1.15630758e+10 1.10452744e+10 1.12657708e+10]
'EJ2': parameters: qubit_id (),:[1, 11] (11 points) without setter
data: [1.94353002e+09 1.67318346e+09 1.91147514e+09 1.76947753e+09
 1.81072654e+09 1.60795669e+09 1.98889801e+09 2.18918806e+09
 2.00107198e+09 1.94107336e+09 1.94564994e+09]
'fr': parameters: qubit_id (),:[1, 11] (11 points) without setter
data: [6.2600e+09 6.3070e+09 6.3623e+09 6.4140e+09 6.4587e+09 6.4984e+09
 6.5550e+09 6.6060e+09 6.6580e+09 6.7110e+09 6.7630e+09]
'phi0': parameters: qubit_id (),:[1, 11] (11 points) without setter
data: [0.08181627 0.08663242 0.10045189 0.08639362 0.08660264 0.08617179
 0.0791041  0.09000315 0.07698168 0.08347794 0.0748904 ]
'L': parameters: coil_id (),:[1, 11]

In [33]:
load_parameters_dict(exdir_db_inst)

{'inductance_matrix_type': 'chain-nn',
 'qubit_resonator_individual': 'equal_claws',
 'qubit_qubit_coupling': 'alternating-chain-nn',
 'EC': '161574581.1907917',
 'g': '586.6709956926836',
 'J1': '0.011727299185763845',
 'J2': '0.003000045037643214',
 'EJ1': {'1': 11816223384.264452,
  '2': 11168950261.446684,
  '3': 11815374828.756998,
  '4': 11263129085.763647,
  '5': 10981552502.675322,
  '6': 10909809335.89073,
  '7': 11666221426.430653,
  '8': 10903644122.926136,
  '9': 11563075836.984825,
  '10': 11045274424.134794,
  '11': 11265770833.103388},
 'EJ2': {'1': 1943530015.2166,
  '2': 1673183461.373451,
  '3': 1911475144.5540576,
  '4': 1769477531.8836436,
  '5': 1810726542.9254663,
  '6': 1607956688.359111,
  '7': 1988898006.2197614,
  '8': 2189188060.0125456,
  '9': 2001071976.5662835,
  '10': 1941073364.3633077,
  '11': 1945649944.4130301},
 'fr': {'1': 6260000000.0,
  '2': 6307000000.0,
  '3': 6362300000.0,
  '4': 6414000000.0,
  '5': 6458700000.0,
  '6': 6498400000.0,
  '7': 65